# Convertitore da FASTQ a FASTA
**Avenoso Luigi 844787**

##### Testo esercizio
Si richiede di scrivere un convertitore da FASTQ a FASTA che prenda in input un file di reads in formato FASTQ e produca in formato FASTA i soli reads che hanno le seguenti caratteristiche: (1) non sono più corti di una soglia L1 e non sono più lunghi di una soglia L2, (2) la qualità minima delle basi supera una soglia Q1, (3) contengono una sottoregione con qualità minima Q2 (maggiore di Q1) che è lunga almeno P% della lunghezza del read.
Viene richiesto l'uso di Biopython per leggere i reads dal file FASTQ in input e per stampare (in standard output o su file) i reads in formato FASTA.
L1, L2 (> L1), Q1, Q2 (> Q1) e P devono essere parametri in input.
Per ognuno dei reads in output, l'header FASTA deve contenere le seguenti informazioni: (1) identificatore, (2) lunghezza, (3) qualità minima delle basi, (4) start ed end della sottoregione con qualità minima Q2, (5) qualità media della sottoregione con qualità minima Q2
Il convertitore può essere prodotto sia come script che come Jupyter Notebook, deve essere adeguatamente commentato e deve essere caricato in un repository GitHub di cui va consegnato il link

In [1]:
# Richiede Biopython
# !conda install -y -c conda-forge biopython

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from numpy import mean

_get_subinterval(quality_string, q2)_
- Ritorna la posizione di inizio e fine della sottoregione

_fake_interval(list_pos)_
- Ritorna True se è presente il fake interval, False altrimenti

**Nota:** il fake interval è [1,1] e significa che tutti i valori in `bool_subquality` sono uguali a `False` (cioé nella stringa qualità tutti i valori sono al di sotto della soglia minima richiesta)

In [3]:
def get_subinterval(quality_string, q2):
    bool_subquality = [qlt >= q2 for qlt in quality_string]
    
    start_list = []
    for i in range(len(bool_subquality))[1:]:
        if bool_subquality[i] == True and bool_subquality[i-1] == False:
            start_list.append(i)
        
    if bool_subquality[0] == True:
        start_list[:0] = [0]
        
    start_list[:0] = [1]
    
    end_list = []
    for j in range(len(bool_subquality))[:-1]:
        if bool_subquality[j] == True and bool_subquality[j+1] == False:
            end_list.append(j)
        
    if bool_subquality[-1] == True:
        end_list[len(bool_subquality):] = [len(bool_subquality)-1]
        
    end_list[:0] = [0]
    
    interval_lengths = [end_list[i] - start_list[i] + 1 for i in range(len(start_list))]
    
    index = interval_lengths.index(max(interval_lengths))
    interval_start = start_list[index]
    interval_end = end_list[index]
    
    return [interval_start, interval_end+1]


def fake_interval(list_pos):
    return list_pos[0] == 1 and list_pos[1] == 1

Input parametri

In [4]:
L1 = L2 = Q1 = P = -1

while L1 <= 0:
    try:
        L1 = int(input("Soglia L1 (min):"))
        if L1 <= 0:
            print("Deve essere un numero intero > 0")
    except ValueError:
        print("Deve essere un numero intero")


while L2 <= 0 or L1 >= L2:
    try:
        L2 = int(input("Soglia L2 (max):"))
        if L2 <= 0 or L1 >= L2:
            print("Deve essere > 0 e strettamente maggiore di L1")
    except ValueError:
        print("Deve essere un numero intero")
    
intero = False

while not intero:
    try:
        Q1 = int(input("Qualità minima Q1:"))
        intero = True
    except ValueError:
        print("Deve essere un numero intero")
        intero = False

Q2 = Q1
    
while Q1 >= Q2:
    try:
        Q2 = int(input("Qualità minima Q2:"))
        if Q1 >= Q2:
            print("Deve essere un numero intero strettamente maggiore di Q1")
    except ValueError:
        print("Deve essere un numero intero")

        
        
while P <= 0 or P > 100:
    try:
        P = int(input("Soglia percentuale P:"))
        if P <= 0 or P > 100:
            print("Deve essere un numero intero tra 1 e 100")
    except ValueError:
        print("Deve essere un numero intero")


Soglia L1 (min):20
Soglia L2 (max):40
Qualità minima Q1:35
Qualità minima Q2:40
Soglia percentuale P:50


In [5]:
# Lettura file fastq e creazione lista con il contenuto del file letto

file_name = 'input.fq'
fastq_records = SeqIO.parse(file_name, 'fastq')
fastq_record_list = list(fastq_records)

In [6]:
# Stringa per le read che rispettano i vincoli
seq_to_file = ''

# Controllo dell'intera lista di record
for read in fastq_record_list:
    length_read = len(read.seq)
    
    if length_read >= L1 and length_read <= L2:
        check_quality = [qlt >= Q1 for qlt in read.letter_annotations['phred_quality']]
        if not False in check_quality:
            length_subread_att = int(P * len(read.seq) / 100)
            interval = get_subinterval(read.letter_annotations['phred_quality'], Q2)
            length_sub_seq = interval[1] - interval[0]
            if length_sub_seq >= length_subread_att and not fake_interval(interval):
                ann_seq = SeqRecord(read.seq)
                ann_seq.id = read.id 
                min_qlt = str(min(read.letter_annotations['phred_quality']))
                start = interval[0]
                end = interval[1]
                subseq_qlt = read.letter_annotations['phred_quality'][start:end]
                media = str(mean(subseq_qlt)) 
                ann_seq.description = 'len:' + str(len(read.seq)) + ' min_quality:' + min_qlt + ' start:' + str(start) + ' end:' + str(end) + ' mean_sub_seq:' + media
                seq_to_file = seq_to_file + ann_seq.format("fasta")

In [7]:
# Scrittura su file fasta se almeno un read ripsetta i vincoli 

if seq_to_file != '':
    file_fasta = open('output.fa', 'wt')
    file_fasta.write(seq_to_file.format("fasta"))
    file_fasta.close()
    print("Creato file output.fa")
else:
    print("Nessun reads rispetta i vincoli")

Creato file output.fa
